In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
from datasets import *
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

In [ ]:
dataset = load_dataset("VietTung04/MovieReviewsData")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 40000
    })
})

In [ ]:
model = TFAutoModel.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Model's input Representation
inputs = tokenizer(
    ['This is the first sentence of the list', 'And this is the second sentence'],
    padding='max_length',
    truncation=True,
    return_tensors='tf'
)
inputs

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[ 101, 2023, 2003, ...,    0,    0,    0],
       [ 101, 1998, 2023, ...,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
# Model's output Representation
output = model(inputs)
output

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 512, 768), dtype=float32, numpy=
array([[[-0.19846863,  0.1220593 ,  0.07143681, ..., -0.28404954,
          0.25335237,  0.67353266],
        [-1.1035013 , -0.20050868,  0.06712086, ..., -0.42106104,
          0.7564434 ,  0.24117102],
        [-0.41054422, -0.4783618 ,  0.56401414, ...,  0.13063228,
          0.13307247,  0.81707734],
        ...,
        [-0.49991387, -0.15074179,  0.2647283 , ...,  0.47119293,
          0.11211342,  0.03264038],
        [-0.3567574 ,  0.00444418,  0.43286303, ...,  0.35083777,
          0.03900161,  0.01557245],
        [-0.7697736 , -0.20203887,  0.3417852 , ...,  0.4862639 ,
          0.19865029,  0.05438824]],

       [[-0.21683055,  0.23335598,  0.04053709, ..., -0.1827562 ,
          0.4949286 ,  0.61166245],
        [-0.19732277, -0.09921859,  0.05200794, ..., -0.10645243,
          0.8554506 ,  0.13572574],
        [-0.93401885, -0.30923012,  0.4507844 , .

In [ ]:
train_testvalid = dataset['train'].train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']}
)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
})

In [ ]:
def tokenize(batch):
  return tokenizer(batch['text'], padding='max_length', truncation=True)

In [ ]:
ds_encoded = ds.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
ds_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
})

In [ ]:
ds_encoded.set_format(
    'tf', # Convert these to tensor
    columns=['input_ids', 'attention_mask', 'token_type_ids', 'label']
)

batch_size = 8

def order(inputs):
  data = list(inputs.values())
  return {
      'input_ids': data[1],
      'attention_mask': data[2],
      'token_type_ids': data[3]
  }, data[0]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(ds_encoded['train'][:])
train_dataset = train_dataset.batch(batch_size).shuffle(1000)
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
eval_dataset = tf.data.Dataset.from_tensor_slices(ds_encoded['valid'][:])
eval_dataset = eval_dataset.batch(batch_size)
eval_dataset = eval_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(ds_encoded['test'][:])
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
next(iter(train_dataset))

({'input_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
  array([[  101,  2928, 14255, ...,     0,     0,     0],
         [  101,  2023,  2001, ...,     0,     0,     0],
         [  101,  4151, 10231, ...,     0,     0,     0],
         ...,
         [  101,  1045,  2052, ...,     0,     0,     0],
         [  101,  1996,  5409, ...,     0,     0,     0],
         [  101,  1045,  2179, ...,     0,     0,     0]])>,
  'attention_mask': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])>,
  'token_type_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0

In [ ]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
classifier = BERTForClassification(model, num_classes=2)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
import os
checkpoint_path = "checkpoint_/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
history = classifier.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=3,
    callbacks=[cp_callback]
)

Epoch 1/3
4000/4000 [==============================] - ETA: 0s - loss: 0.2387 - accuracy: 0.9003
Epoch 1: saving model to checkpoint_/cp.ckpt
4000/4000 [==============================] - 4178s 1s/step - loss: 0.2387 - accuracy: 0.9003 - val_loss: 0.1908 - val_accuracy: 0.9295
Epoch 2/3
4000/4000 [==============================] - ETA: 0s - loss: 0.1275 - accuracy: 0.9545
Epoch 2: saving model to checkpoint_/cp.ckpt
4000/4000 [==============================] - 4118s 1s/step - loss: 0.1275 - accuracy: 0.9545 - val_loss: 0.2003 - val_accuracy: 0.9275
Epoch 3/3
4000/4000 [==============================] - ETA: 0s - loss: 0.0685 - accuracy: 0.9767
Epoch 3: saving model to checkpoint_/cp.ckpt
4000/4000 [==============================] - 4157s 1s/step - loss: 0.0685 - accuracy: 0.9767 - val_loss: 0.2105 - val_accuracy: 0.9355


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
classifier.save('my_model.keras')

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
classifier.evaluate(test_dataset)

500/500 [==============================] - 163s 326ms/step - loss: 0.2032 - accuracy: 0.9370


[0.20324388146400452, 0.9369999766349792]

In [ ]:
def predict_sentiment(text):
  # Text the sentence -> Process it to be BERT's input -> convert the inputs to tensor -> call model
  inputs = tokenizer(
    [text],
    padding='max_length',
    truncation=True,
    return_tensors='tf'
  )
  output = classifier.call(inputs).numpy()[0]
  label = np.argmax(output)
  confidence = output[label]
  return label, confidence

sentence = "This is a test sentence which feels very good."
label, confidence = predict_sentiment(sentence)
print(f"Sentence: {sentence}")
print(f"Label: {label} ({'Negative' if label == 0 else 'Positive'}) with confidence score {confidence: .2f}.")

Sentence: This is a test sentence which feels very good.
Label: 1 (Positive) with confidence score  0.98


In [ ]:
# Check with real dataset
for i in range(10):
  text = dataset['train']['text'][i]
  real_label = dataset['train']['label'][i]
  predicted_label = predict_sentiment(text)[0]
  print(f"Sentence no.{i} has label {real_label} -> model predicted {predicted_label}.")

Sentence no.0 has label 1 -> model predicted 1.
Sentence no.1 has label 0 -> model predicted 0.
Sentence no.2 has label 1 -> model predicted 1.
Sentence no.3 has label 1 -> model predicted 1.
Sentence no.4 has label 1 -> model predicted 1.
Sentence no.5 has label 0 -> model predicted 0.
Sentence no.6 has label 0 -> model predicted 0.
Sentence no.7 has label 1 -> model predicted 1.
Sentence no.8 has label 0 -> model predicted 0.
Sentence no.9 has label 1 -> model predicted 1.


In [ ]:
submission = pd.read_csv("test_data.csv")
submission.head(5)

,Id,text
0,0,What can possibly said about this movie other ...
1,1,I dont care how many bad reviews purple rain g...
2,2,Ken Russell directed this weird Not very ero...
3,3,This is a great movie from the lost age of rea...
4,4,I have a problem with the movie snobs who cons...


In [ ]:
submission['Category'] = submission['text'].apply(lambda x: predict_sentiment(x)[0])
submission.head(5)

,Id,text,Category
0,0,What can possibly said about this movie other ...,0
1,1,I dont care how many bad reviews purple rain g...,1
2,2,Ken Russell directed this weird Not very ero...,0
3,3,This is a great movie from the lost age of rea...,1
4,4,I have a problem with the movie snobs who cons...,1


In [ ]:
submission = submission.drop('text', axis=1)

In [ ]:
submission.to_csv('submission.csv', index=False)